<a href="https://colab.research.google.com/github/shinokazu/DeepLabCut/blob/master/Colab_TrainNetwork_VideoAnalysis_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species
https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- analyze a novel video

###This notebook assumes you already have a project folder with labeled data! 

Note, if you used WINDOWS to create the project and label, you must run "create_training_dataset" on Colaboratory. Now (in DeepLabCut 2.0.4) our code will automatically detect if you used Windows for project creation, and convert the files appropriately for using on Linx! 


## Let's look at info about the Colab Environment (and decide, GPU or TPU):

First, go to "Runtime" ->"change runtime type"->select "Python3", and then decide, "GPU or TPU!""


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


If you want to use a GPU:

In [2]:
import tensorflow as tf
tf.__version__

'1.13.0-rc2'

In [3]:
#let's make sure we see a GPU:
tf.test.gpu_device_name()

#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

'/device:GPU:0'

If you selected to use a TPU: 

In [0]:
#let's make sure we see a TPU:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

## Link your Google Drive (with your labeled data, or the demo data):

### First, place your porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [4]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
#(this will take a few minutes to install all the dependences!)

!pip install deeplabcut

    100% |████████████████████████████████| 143kB 14.3MB/s 
    100% |████████████████████████████████| 3.3MB 9.6MB/s 
    100% |████████████████████████████████| 7.4MB 5.5MB/s 
    100% |████████████████████████████████| 737kB 26.5MB/s 
    100% |████████████████████████████████| 51kB 22.1MB/s 
    100% |████████████████████████████████| 215kB 30.8MB/s 
    100% |████████████████████████████████| 26.2MB 1.6MB/s 
    100% |████████████████████████████████| 552kB 22.1MB/s 
    100% |████████████████████████████████| 4.9MB 8.6MB/s 
    100% |████████████████████████████████| 3.8MB 10.4MB/s 
    100% |████████████████████████████████| 25.3MB 1.8MB/s 
    100% |████████████████████████████████| 276kB 27.3MB/s 
    100% |████████████████████████████████| 184kB 30.2MB/s 
    100% |████████████████████████████████| 706kB 21.5MB/s 
    100% |████████████████████████████████| 102kB 31.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/96/96/ba/ad15a750d843219c0578efe536dbb896308d70d87dc7a35c

In [6]:
#These are some Colab specific work-arounds, but they work! (typically not required, as they are installed with "pip install deeplabcut")

!pip install Pillow==4.0.0

from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

!pip install ruamel.yaml==0.15
!pip install pandas==0.21.0

In [0]:
#GUIs don't work on the cloud, so label your data locally on your computer! 
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

In [29]:
#Be sure you have a copy of your project folder in your Google Drive. Then, let's go to your deeplabcut folder!
%cd /content/drive/My Drive/DeepLabCut-master/

/content/drive/My Drive/DeepLabCut-master


In [30]:
import deeplabcut

#sometimes the backend kernel is defaulting to something other than what we want
#if you see the warnings, you can re-run this cell and they go way, as we re-set the backend! :) 

#possible warning--> UserWarning: This call to matplotlib.use() has no effect because the backend has already
#been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot, or matplotlib.backends is imported for the first time.

DLC loaded in light mode; you cannot use the labeling GUI!
Project loaded in colab-mode. Apparently Colab has trouble loading statsmodels, so the smooting & outlier frame extraction is disabled. Sorry!


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!


In [0]:
#create a path variable that links to your google drive copy:
path_config_file = '/content/drive/My Drive/DeepLabCut-master/examples/Reaching-Mackenzie-2018-08-30/config.yaml' #change to yours!

## Create a training dataset:
### You must do this step inside of Colab:
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [0]:
# If you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models! 
#Then, run this cell. 
deeplabcut.create_training_dataset(path_config_file)

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [0]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [0]:
%matplotlib inline
deeplabcut.evaluate_network(path_config_file)#,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- pplease see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [0]:
videofile_path = ['/content/drive/My Drive/DeepLabCut/examples/Reaching-Mackenzie-2018-08-30/videos/MovieS2_Perturbation_noLaser_compressed.avi'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
#for making interactive plots.
%matplotlib notebook 
deeplabcut.plot_trajectories(path_config_file,videofile_path)